In [ ]:
import requests
from dotenv import load_dotenv
import os

# 환경변수 불러오기
load_dotenv()

# 환경변수에서 키, 엔드포인트 불러오기
endpoint_base = os.getenv("clu_endpoint")  
api_key = os.getenv("clu_key")
api_id = os.getenv("clu_id")
project_name = "PPCLU"
deployment_name = "ppv1-test"


# 요청 헤더
headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/json",
    "Apim-Request-Id" : api_id
}

# 요청 본문 (Query)
query = {
    "kind": "Conversation",
    "analysisInput": {
        "conversationItem": {
            "id": "1",
            "participantId": "user",
            "text": "예산은 3천만 원 정도 있고, 서울에서 3월 말쯤 뮤지컬 하나 해보고 싶어요",
            "modality": "text",
            "language": "ko"
        }
    },
    "parameters": {
        "projectName": project_name,
        "deploymentName": deployment_name,
        "verbose": True,
        "stringIndexType": "TextElement_V8"
    }
}

# 응답 보기 좋게 출력하는 함수
def pretty_print_clu_result(clu_response):
    try:
        result = clu_response['result']
        prediction = result['prediction']
        intent = prediction['topIntent']
        entities = prediction.get('entities', [])

        print(f'당신: {query["analysisInput"]["conversationItem"]["text"]}')
        print(" 추출된 엔터티:")
        for ent in entities:
            category = ent['category']
            text = ent['text']
            score = ent.get("confidenceScore", 0)
            print(f"- {category}: {text} (신뢰도: {score:.2f})")
    except Exception as e:
        print(" 오류 발생:", e)

# API 호출
response = requests.post(
    endpoint_base,
    headers=headers,
    json=query
)

# 결과 저장
if response.status_code == 200:
    clu_response = response.json()  #  저장
    pretty_print_clu_result(clu_response)  # 출력도 계속 하고
else:
    print(f" 요청 실패: {response.status_code}")
    print(response.text)
    clu_response = None  # 실패 시에도 변수 정의

# 클루 결과 객체를 그대로 다음 파이프라인으로 넘기기 위해 출력
clu_response  # Jupyter Notebook에서 마지막 줄에 출력



당신: 예산은 3천만 원 정도 있고, 서울에서 3월 말쯤 뮤지컬 하나 해보고 싶어요
 추출된 엔터티:
- 예산: 3천만 원 (신뢰도: 1.00)
- 지역: 서울 (신뢰도: 1.00)
- 일정: 3월 말 (신뢰도: 1.00)
- 장르: 뮤지컬 (신뢰도: 1.00)


{'kind': 'ConversationResult',
 'result': {'query': '예산은 3천만 원 정도 있고, 서울에서 3월 말쯤 뮤지컬 하나 해보고 싶어요',
  'prediction': {'topIntent': '공연_기획',
   'projectKind': 'Conversation',
   'intents': [{'category': '공연_기획', 'confidenceScore': 1},
    {'category': 'None', 'confidenceScore': 0}],
   'entities': [{'category': '예산',
     'text': '3천만 원',
     'offset': 4,
     'length': 5,
     'confidenceScore': 1},
    {'category': '지역',
     'text': '서울',
     'offset': 17,
     'length': 2,
     'confidenceScore': 1},
    {'category': '일정',
     'text': '3월 말',
     'offset': 22,
     'length': 4,
     'confidenceScore': 1},
    {'category': '장르',
     'text': '뮤지컬',
     'offset': 28,
     'length': 3,
     'confidenceScore': 1}]}}}